In [ ]:
!pip install pymorphy2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 522.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import re

import gensim
import pymorphy2

import nltk
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize, sent_tokenize

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/Авалон/Диплом/data')

Mounted at /content/drive


In [ ]:
nltk.download('wordnet')
nltk.download('punkt')

stop_words = \
    ['и', 'в', 'во',  'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
     'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от',
     'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже',
     'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там',
     'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы',
     'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под',
     'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь',
     'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех',
     'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот',
     'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя',
     'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой',
     'им', 'более', 'всегда', 'конечно', 'всю', 'между','не', ]
stop_words.extend(['очень', 'ооочень', 'это', 'данное'])

# Загрузка данных

In [ ]:
df = pd.read_csv('coffee.csv')
df['rating'] = df['rating'].astype('float16')
df_tmp = df['text']


# Токенизация coffee.csv и лемматизация



## На предложения

In [ ]:
print("Токенизация отзывов на предложения  -- ", end='')
split_regex = re.compile(r'[.|!|?|…|(|)]')
my_sentences = [filter(lambda t: t, [t.strip() for t in split_regex.split(text_x.replace(r'\n', ''))]) for text_x in df_tmp]
print("ОК")
# итератор, вызывается только один раз
#for s in sentences[0]:
#    print(s)
# nltk.download('wordnet')# Нужно

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# токенизация по словам (с удалением пунктуации)
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

phrases_tokens = []
for phrase in tqdm(my_sentences, desc='Токенизация предложений на слова'):
    words = list(sent_to_words(phrase))
    phrases_tokens.append(words)

morph = pymorphy2.MorphAnalyzer()
# по предложениям
result = []
for doc_x in tqdm(phrases_tokens, desc='Токенизация лемматизация по предложениям'):
  result_doc = []
  for sentence_x in doc_x:
    result_sent = []
    for word_x in sentence_x:
      word_tmp = morph.parse(word_x)[0].normal_form
      if word_tmp not in stop_words:
        result_sent.append(word_tmp)

    result_doc.append(result_sent)
  result.append(result_doc)

Токенизация отзывов на предложения  -- ОК


Токенизация лемматизация по предложениям: 100%|██████████| 117818/117818 [17:56<00:00, 109.48it/s]


In [ ]:
result[0]

[['глубинка', 'страна', 'весь', 'свой', 'проявление'],
 ['ассортимент',
  'столовския',
  'интерьер',
  'качество',
  'цена',
  'приемлемый',
  'среднея',
  'бюджетноить',
  'столовой',
  'ссср',
  'всё',
  'чистый',
  'всё',
  'новый'],
 ['хотеть', 'трасса'],
 ['поесть', 'желудок', 'бастовать', 'значит', 'риск', 'оправдать'],
 ['номер', 'ночлег', 'аналогичный', 'толк'],
 ['пластиковый', 'окно', 'нормально', 'закрыться', 'штора', 'окно', 'тюль'],
 ['первый', 'этаж', 'плохо'],
 ['мало', 'заглядывать'],
 ['туалет', 'душа'],
 ['совка', 'повториться', 'трасса', 'руб', 'соидти']]

## На фразы

In [ ]:
print("Токенизация отзывов на предложения  -- ", end='')
split_regex = re.compile(r'[.|!|?|…|(|)|,]')
my_sentences = [filter(lambda t: t, [t.strip() for t in split_regex.split(text_x.replace(r'\n', ''))]) for text_x in df_tmp]
print("ОК")
# итератор, вызывается только один раз
#for s in sentences[0]:
#    print(s)
# nltk.download('wordnet')# Нужно

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# токенизация по словам (с удалением пунктуации)
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

phrases_tokens = []
for phrase in tqdm(my_sentences, desc='Токенизация предложений на слова'):
    words = list(sent_to_words(phrase))
    phrases_tokens.append(words)

morph = pymorphy2.MorphAnalyzer()
# по предложениям
result_phrases = []
for doc_x in tqdm(phrases_tokens, desc='Токенизация лемматизация по предложениям'):
  result_doc = []
  for sentence_x in doc_x:
    result_sent = []
    for word_x in sentence_x:
      word_tmp = morph.parse(word_x)[0].normal_form
      if word_tmp not in stop_words:
        result_sent.append(word_tmp)

    result_doc.append(result_sent)
  result_phrases.append(result_doc)


Токенизация отзывов на предложения  -- ОК


Токенизация лемматизация по предложениям: 100%|██████████| 117818/117818 [17:41<00:00, 110.98it/s]


In [ ]:
result[0]

# Создание DataFrame

In [ ]:
index_x = 0
result_dict = {}

for otsiv, my_list in tqdm(zip(df.index, result), desc="Создание df"):
  for word_x in my_list:
    word_x_tmp = " ".join(word_x)
    t = pd.concat([pd.Series(otsiv),pd.Series(word_x_tmp)])
    result_dict[index_x] = t
    index_x += 1

sentenses = pd.DataFrame(result_dict).transpose()

Создание df: 117818it [04:34, 429.10it/s]


In [ ]:
index_x = 0
result_dict = {}

for otsiv, my_list in tqdm(zip(df.index, result_phrases), desc="Создание df"):
  for word_x in my_list:
    word_x_tmp = " ".join(word_x)
    t = pd.concat([pd.Series(otsiv),pd.Series(word_x_tmp)])
    result_dict[index_x] = t
    index_x += 1

phrases = pd.DataFrame(result_dict).transpose()

Создание df: 117818it [07:54, 248.35it/s]


In [ ]:
sentenses.to_csv("sentences.csv", index=False)

In [ ]:
phrases.to_csv("phrases.csv", index=False)

In [ ]:
pd.DataFrame({result_num: {num_x: " ".join(result_text_x) for num_x, result_text_x in enumerate(result_text)} for result_num, result_text in enumerate(result)}).transpose()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,глубинка страна весь свой проявление,ассортимент столовския,интерьер,качество цена приемлемый среднея бюджетноить с...,всё чистый,всё новый,хотеть трасса,поесть,желудок бастовать,значит риск оправдать,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,пить кофе рим,париж,вкусный,капучий фундучный молоко фирменный сливка джинж,пробовать,десерт необычный,ребята барист больший молодец,улучшить,маловатый место,посадкой придумать,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,удобный расположение,метро идти мина быстрый шаг промзона,кофе место мало,желать,пирожное вкусный,кофе,бывать вкусный,второй именно туда поида,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,самый большоя плюс,набережная,шикарный вид море,красиво,уютный,собственно плюс закончиться,огорчать отношение посетитель,официант неприветливый,здравствуит,свидание,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,добрый день,сегодня второй посетить дочкой ресторан,нравиться меню,куриный суп лапша,пицца сыр,яблочный штрудель сацивить курица,облепиховыя чай,успеть попробовать посещение,особенно порадовать прекрасный обслуживание,спасибо алла забота,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,прекрасный заведение,быстрый подача блюдо,приветливый официант,готовить замечательный,хорошие выбор хмельный напиток,классика малиновый саур обязательно прийти туд...,здорово,чаевой оставить картой,каждый официант свой карточка кюар код,музыка,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,отличный заведение,ранний ремонт уютный,атмосферный,сеичас народ казаться маленький,сервис обслуживание высота,пиво свежий,считать вернуть меню варёный пельмень горшочек,рекомендовать,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,среднея оценка магазин,выбор никакой,купить,приходиться идти сюда,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,несколько год заказывать пицца,вкусный,просто пальчик облизать,расстроиться заказать,пицца,гренка,картошка фри,сожаление пицца ждать сюрприз металлический ст...,благо время поздно ребёнок спасть успеть попро...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
